In [1]:
import os
from bs4 import BeautifulSoup
import math
import numpy as np
import shapely as shp
import pandas as pd
import geopandas as gpd
# import matplotlib.pyplot as plt
import rasterio as rio
import rasterio.mask

In [2]:
cwd = "c:\\Users\\m1865\\Desktop\\DISC"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

In [3]:
df_Site = pd.read_csv(cwd + "//Site.csv")
df_Site.head()

,Number,Site,Serial Number (FLOX),Country,Location,Latitude,Longitude,Priority,Plant functional type (PFT),Reference network,Number of components,Reduce ROI size (600 x 600 m),Move ROI,Comments,PRISMA Availability (900m Buffer ROI Check),PRISMA Earliest Date,PRISMA Latest Date (Before 2024-06-26),Tomi,S2 Cloud
0,1,ATGE,NaN,NaN,NaN,52.466778,12.959778,NaN,NaN,HYPERNET,NaN,NaN,NaN,NaN,2,2022-05-06,2023-08-14,Y,NaN
1,2,ATLAS-Mohammed V,NaN,Morocco,ATLAS-Mohammed V,33.406152,-5.103319,Low,NaN,NaN,NaN,Yes,Yes,"mixed pixel, complex topography",2,2023-11-23,2023-11-23,NaN,NaN
2,3,AU-MIEMING,JB-042-GW,Austria,Austria,47.316700,10.970300,NaN,NaN,FLOX,NaN,No,No,topography?,0,NaN,NaN,Duplicate,NaN
3,4,BASP,NaN,NaN,NaN,39.049139,-2.075917,NaN,NaN,HYPERNET,NaN,NaN,NaN,NaN,27,2022-05-09,2024-06-18,Y,NaN
4,5,BE-BRASCHAAT,JB-021-SW,Belgium,Brasschaat,51.307600,4.519900,NaN,NaN,FLOX,NaN,No,No,Check the site with Sentinal 3,7,2020-09-12,2022-06-01,NaN,NaN


In [4]:
for i in range(65,df_Site.shape[0]):
    site_Name = df_Site["Site"][i]
    site_Lat = df_Site.loc[i,"Latitude"]
    site_Lon = df_Site.loc[i,"Longitude"]
    if df_Site.loc[i,"Reduce ROI size (600 x 600 m)"] == "Yes":
        site_Reduced = True
    else:
        site_Reduced = False
    df_4326 = pd.DataFrame({
        "Site": [site_Name],
        "Latitude": [site_Lat],
        "Longitude": [site_Lon]
    })
    # Create a point shapefile based on the site, using Lon-Lat
    gdf_4326 = gpd.GeoDataFrame(
        df_4326,
        geometry = gpd.points_from_xy(df_4326['Longitude'], df_4326['Latitude']),
        crs = "EPSG:4326"
    )
    for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L1C"):
        for name in files:
            temp = os.path.join(path, name)
            if "IMG_DATA" in temp and temp[-3:] == 'jp2' and "B08" in temp:
                # print(temp)
                path_L1C_B08_raw = temp
    for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L1C"):
        for name in files:
            temp = os.path.join(path, name)
            if "MTD_DS.xml" in temp:
                # print(temp)
                path_L1C_xml_DS = temp
    for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L1C"):
        for name in files:
            temp = os.path.join(path, name)
            if "MTD_TL.xml" in temp:
                # print(temp)
                path_L1C_xml_TL = temp
    for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L2A"):
        for name in files:
            temp = os.path.join(path, name)
            if temp[-3:] == 'jp2'in temp and "10m" in temp and "B04" in temp :
                path_L2A_B04_raw = temp
            if temp[-3:] == 'jp2'in temp and "10m" in temp and "B08" in temp :
                path_L2A_B08_raw = temp
    for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L2A"):
        for name in files:
            temp = os.path.join(path, name)
            if "MTD_DS.xml" in temp:
                # print(temp)
                path_L2A_xml_DS = temp
    image_L1C_B08 = rio.open(path_L1C_B08_raw)
    image_L2A_B04 = rio.open(path_L2A_B04_raw)
    image_L2A_B08 = rio.open(path_L2A_B08_raw)
    values_L1C_B08 = image_L1C_B08.read(1)
    values_L2A_B04 = image_L2A_B04.read(1)
    values_L2A_B08 = image_L2A_B08.read(1)
    crs_L1C = image_L1C_B08.crs.data["init"].split(":")[1]
    crs_L2A = image_L2A_B04.crs.data["init"].split(":")[1]
    # print("The EPSG of L1C is " + crs_L1C)
    # print("The EPSG of L2A is " + crs_L2A)
    # In the case that L1C and L2A have different crs, give an error. But this shouldn't happen. 
    if crs_L2A != crs_L1C:
        raise SystemExit("Stop right there!")
    crs_Final = 'EPSG:' + crs_L1C
    gdf_New = gdf_4326.copy()
    gdf_New = gdf_New.to_crs(crs_Final)
    site_x = gdf_New.geometry.x.values[0]
    site_y = gdf_New.geometry.y.values[0]
    site_row, site_col = image_L2A_B08.index(site_x, site_y)
    site_pixel_x, site_pixel_y = image_L2A_B08.xy(site_row, site_col)
    # Get the coordinates of the four corners
    # 10m
    site_x_left_10m = site_pixel_x - 5
    site_x_right_10m = site_pixel_x + 5
    site_y_top_10m = site_pixel_y + 5
    site_y_bottom_10m = site_pixel_y - 5
    # 30m
    site_x_left_30m = site_pixel_x - 15
    site_x_right_30m = site_pixel_x + 15
    site_y_top_30m = site_pixel_y + 15
    site_y_bottom_30m = site_pixel_y - 15
    # 100m
    site_x_left_100m = site_pixel_x - 55
    site_x_right_100m = site_pixel_x + 55
    site_y_top_100m = site_pixel_y + 55
    site_y_bottom_100m = site_pixel_y - 55
    # 100 * 1.5 = 150m
    site_x_left_150m = site_pixel_x - 75
    site_x_right_150m = site_pixel_x + 75
    site_y_top_150m = site_pixel_y + 75
    site_y_bottom_150m = site_pixel_y - 75
    # 300m
    site_x_left_300m = site_pixel_x - 155
    site_x_right_300m = site_pixel_x + 155
    site_y_top_300m = site_pixel_y + 155
    site_y_bottom_300m = site_pixel_y - 155
    # 300 * 1.5 = 450m
    site_x_left_450m = site_pixel_x - 225
    site_x_right_450m = site_pixel_x + 225
    site_y_top_450m = site_pixel_y + 225
    site_y_bottom_450m = site_pixel_y - 225
    if site_Reduced:
        # 600m
        site_x_left_600m = site_pixel_x - 305
        site_x_right_600m = site_pixel_x + 305
        site_y_top_600m = site_pixel_y + 305
        site_y_bottom_600m = site_pixel_y - 305
    # 900m
    site_x_left_900m = site_pixel_x - 455
    site_x_right_900m = site_pixel_x + 455
    site_y_top_900m = site_pixel_y + 455
    site_y_bottom_900m = site_pixel_y - 455
    # 900 * 1.5 = 1350m
    site_x_left_1350m = site_pixel_x - 675
    site_x_right_1350m = site_pixel_x + 675
    site_y_top_1350m = site_pixel_y + 675
    site_y_bottom_1350m = site_pixel_y - 675
    # 1200m
    site_x_left_1200m = site_pixel_x - 605
    site_x_right_1200m = site_pixel_x + 605
    site_y_top_1200m = site_pixel_y + 605
    site_y_bottom_1200m = site_pixel_y - 605
    # 1800m
    site_x_left_1800m = site_pixel_x - 905
    site_x_right_1800m = site_pixel_x + 905
    site_y_top_1800m = site_pixel_y + 905
    site_y_bottom_1800m = site_pixel_y - 905
    # 2500m
    site_x_left_2500m = site_pixel_x - 1255
    site_x_right_2500m = site_pixel_x + 1255
    site_y_top_2500m = site_pixel_y + 1255
    site_y_bottom_2500m = site_pixel_y - 1255
    # Now we need to form squares shapefile, which will be the internal area of which we will evaluate the spatial representativeness. 
    shp_10m = shp.box(site_x_left_10m, site_y_bottom_10m, site_x_right_10m, site_y_top_10m)
    gdf_10m = gpd.GeoDataFrame(
        pd.DataFrame({"0": ["0"]}),
        geometry=[shp_10m],
        crs = crs_Final
    )
    shp_30m = shp.box(site_x_left_30m, site_y_bottom_30m, site_x_right_30m, site_y_top_30m)
    gdf_30m = gpd.GeoDataFrame(
        pd.DataFrame({"0": ["0"]}),
        geometry=[shp_30m],
        crs = crs_Final
    )
    shp_100m = shp.box(site_x_left_100m, site_y_bottom_100m, site_x_right_100m, site_y_top_100m)
    gdf_100m = gpd.GeoDataFrame(
        pd.DataFrame({"0": ["0"]}),
        geometry=[shp_100m],
        crs = crs_Final
    )
    shp_150m = shp.box(site_x_left_150m, site_y_bottom_150m, site_x_right_150m, site_y_top_150m)
    gdf_150m = gpd.GeoDataFrame(
        pd.DataFrame({"0": ["0"]}),
        geometry=[shp_150m],
        crs = crs_Final
    )
    shp_300m = shp.box(site_x_left_300m, site_y_bottom_300m, site_x_right_300m, site_y_top_300m)
    gdf_300m = gpd.GeoDataFrame(
        pd.DataFrame({"0": ["0"]}),
        geometry=[shp_300m],
        crs = crs_Final
    )
    shp_450m = shp.box(site_x_left_450m, site_y_bottom_450m, site_x_right_450m, site_y_top_450m)
    gdf_450m = gpd.GeoDataFrame(
        pd.DataFrame({"0": ["0"]}),
        geometry=[shp_450m],
        crs = crs_Final
    )
    if site_Reduced:
        shp_600m = shp.box(site_x_left_600m, site_y_bottom_600m, site_x_right_600m, site_y_top_600m)
        gdf_600m = gpd.GeoDataFrame(
            pd.DataFrame({"0": ["0"]}),
            geometry=[shp_600m],
            crs = crs_Final
        )
    shp_900m = shp.box(site_x_left_900m, site_y_bottom_900m, site_x_right_900m, site_y_top_900m)
    gdf_900m = gpd.GeoDataFrame(
        pd.DataFrame({"0": ["0"]}),
        geometry=[shp_900m],
        crs = crs_Final
    )
    shp_1350m = shp.box(site_x_left_1350m, site_y_bottom_1350m, site_x_right_1350m, site_y_top_1350m)
    gdf_1350m = gpd.GeoDataFrame(
        pd.DataFrame({"0": ["0"]}),
        geometry=[shp_1350m],
        crs = crs_Final
    )
    shp_1200m = shp.box(site_x_left_1200m, site_y_bottom_1200m, site_x_right_1200m, site_y_top_1200m)
    gdf_1200m = gpd.GeoDataFrame(
        pd.DataFrame({"0": ["0"]}),
        geometry=[shp_1200m],
        crs = crs_Final
    )
    shp_1800m = shp.box(site_x_left_1800m, site_y_bottom_1800m, site_x_right_1800m, site_y_top_1800m)
    gdf_1800m = gpd.GeoDataFrame(
        pd.DataFrame({"0": ["0"]}),
        geometry=[shp_1800m],
        crs = crs_Final
    )
    shp_2500m = shp.box(site_x_left_2500m, site_y_bottom_2500m, site_x_right_2500m, site_y_top_2500m)
    gdf_2500m = gpd.GeoDataFrame(
        pd.DataFrame({"0": ["0"]}),
        geometry=[shp_2500m],
        crs = crs_Final
    )
    # Read the DS xml file of L2A
    with open(path_L2A_xml_DS, 'r') as f:
        data = f.read()
    BS_L2A_dS = BeautifulSoup(data, "xml")
    # Get the quantification value! 
    quantification_L2A = int(BS_L2A_dS.find("BOA_QUANTIFICATION_VALUE").text)
    # Get the radiometric offset!
    offset_L2A_B04 = int(BS_L2A_dS.find("BOA_ADD_OFFSET", {"band_id": "3"}).text)
    offset_L2A_B08 = int(BS_L2A_dS.find("BOA_ADD_OFFSET", {"band_id": "7"}).text)
    NDVI = ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A - (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A) / ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A + (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A )
    src = image_L2A_B04
    out_meta = src.meta
    out_meta.update({
        "driver": "GTiff",
        "dtype": 'float64'
    })
    with rio.open(cwd_Images_Processed + "\\" + site_Name + "\\NDVI.tif", 'w', **out_meta) as dest:
        dest.write(NDVI, 1)
    # Read the DS xml file of L1C
    with open(path_L1C_xml_DS, 'r') as f:
        data = f.read()
    BS_L1C_dS = BeautifulSoup(data, "xml")
    # Get the quantification value! 
    quantification_L1C = int(BS_L1C_dS.find("QUANTIFICATION_VALUE").text)
    # Get the radiometric offset!
    offset_L1C = int(BS_L1C_dS.find("RADIO_ADD_OFFSET", {"band_id": "7"}).text)
    # Get the U
    U_L1C = float(BS_L1C_dS.find("U").text)
    # Get the solar irradiance
    SolarIrr = float(BS_L1C_dS.find("SOLAR_IRRADIANCE", {"bandId": "7"}).text)
    # Read the TL xml file of L1C
    with open(path_L1C_xml_TL, 'r') as f:
        data = f.read()
    BS_L1C_dS = BeautifulSoup(data, "xml")
    # Get the sun zenith angle! There should be a 23 x 23 arrays in the xml. Now we save each row as an array and keep all these arrays into a list
    list_SunZenith = []
    for row in BS_L1C_dS.find("Sun_Angles_Grid").find("Zenith").find_all("VALUES"):
        temp_List = row.text.split(" ")
        temp_Arr = np.array(temp_List)
        temp_Arr = temp_Arr.astype(float)
        list_SunZenith.append(temp_Arr)
    # Now we stack these nested-in-list arrays into a 2d array
    index = 0
    for arr in list_SunZenith:
        if index == 0:
            arr_SunZenith = arr
        else:
            arr_SunZenith = np.vstack((arr_SunZenith, arr))
        index = index + 1
    # Get the shape of L1C image, which should be (10980, 10980)
    shape_L1C = values_L1C_B08.shape
    # Repeat each element of sun zenith angle array, in both axies. The final array should have a shape of (11500, 11500)
    arr_SunZenith_Repeat = np.repeat(arr_SunZenith, 500, axis = 1)
    arr_SunZenith_Repeat = np.repeat(arr_SunZenith_Repeat, 500, axis = 0)
    # Index only the first 10980 of each dimension
    arr_SunZenith_Assigned = arr_SunZenith_Repeat[0:shape_L1C[0], 0:shape_L1C[1]]
    # radiance = reflectance * cos(radians(SunZenithAngle)) * solarIrradiance * U / pi
    radiance = (values_L1C_B08 + offset_L1C).astype(float)  * np.cos(np.radians(arr_SunZenith_Assigned)) * SolarIrr / quantification_L1C / (math.pi * (1 / U_L1C))
    NIRv = NDVI * radiance
    src = image_L1C_B08
    out_meta = src.meta
    out_meta.update({
        "driver": "GTiff",
        "dtype": 'float64'
    })
    with rio.open(cwd_Images_Processed + "\\" + site_Name + "\\NIRv.tif", 'w', **out_meta) as dest:
        dest.write(NIRv, 1)
    list_Distance = ['10m','30m','100m','150m','300m','450m','900m','1350m','1200m','1800m','2500m']
    list_GDF = [gdf_10m,gdf_30m,gdf_100m,gdf_150m,gdf_300m,gdf_450m,gdf_900m,gdf_1350m,gdf_1200m,gdf_1800m,gdf_2500m]
    for index in range(len(list_Distance)):
        temp_Distance = list_Distance[index]
        temp_GDF = list_GDF[index]
        src = rio.open(cwd_Images_Processed + "\\" + site_Name + "\\NIRv.tif")
        out_image, out_transform = rio.mask.mask(src, temp_GDF.geometry, crop=True)
        out_meta = src.meta
        out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})

        with rio.open(cwd_Images_Processed + "\\" + site_Name + "\\NIRv " + temp_Distance + ".tif", "w", **out_meta) as dest:
            dest.write(out_image)
    for i in range(len(list_Distance)):
        list_GDF[i].to_file(cwd_Images_Processed + "\\" + site_Name + "\\" + list_Distance[i] + ".shp")
    if site_Reduced:
        src = rio.open(cwd_Images_Processed + "\\" + site_Name + "\\NIRv.tif")
        out_image, out_transform = rio.mask.mask(src, gdf_600m.geometry, crop=True)
        out_meta = src.meta
        out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})

        with rio.open(cwd_Images_Processed + "\\" + site_Name + "\\NIRv 600m.tif", "w", **out_meta) as dest:
            dest.write(out_image)
        gdf_600m.to_file(cwd_Images_Processed + "\\" + site_Name + "\\600m.shp")
    print(site_Name + " finished!")

C:\Users\m1865\AppData\Local\Temp\ipykernel_27360\235916105.py:216: RuntimeWarning: divide by zero encountered in divide
  NDVI = ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A - (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A) / ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A + (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A )
C:\Users\m1865\AppData\Local\Temp\ipykernel_27360\235916105.py:216: RuntimeWarning: invalid value encountered in divide
  NDVI = ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A - (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A) / ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A + (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A )


SP-VAL finished!


C:\Users\m1865\AppData\Local\Temp\ipykernel_27360\235916105.py:216: RuntimeWarning: divide by zero encountered in divide
  NDVI = ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A - (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A) / ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A + (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A )
C:\Users\m1865\AppData\Local\Temp\ipykernel_27360\235916105.py:216: RuntimeWarning: invalid value encountered in divide
  NDVI = ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A - (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A) / ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A + (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A )


SVALBARD finished!


C:\Users\m1865\AppData\Local\Temp\ipykernel_27360\235916105.py:216: RuntimeWarning: divide by zero encountered in divide
  NDVI = ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A - (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A) / ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A + (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A )


...

US-OPE finished!


C:\Users\m1865\AppData\Local\Temp\ipykernel_27360\235916105.py:216: RuntimeWarning: divide by zero encountered in divide
  NDVI = ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A - (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A) / ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A + (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A )


US-SERC finished!
Utqiagvik finished!


C:\Users\m1865\AppData\Local\Temp\ipykernel_27360\235916105.py:216: RuntimeWarning: divide by zero encountered in divide
  NDVI = ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A - (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A) / ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A + (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A )


WWUK finished!
